In [87]:
using ITensors, ITensorMPS, Plots
using Plots.PlotMeasures

In [88]:
function sqr_lat_pbc(Nx::Int, Ny::Int)::Lattice
  N = Nx * Ny
  Nbond = 2N
  latt = Lattice(undef, Nbond)
  b = 0
  for n in 1:N
    x = div(n - 1, Ny) + 1
    y = mod(n - 1, Ny) + 1
    if Nx > 1
      if x < Nx
        latt[b += 1] = LatticeBond(n, n + Ny, x, y, x + 1, y)
      end
      if x == 1
        latt[b += 1] = LatticeBond(n, n + Ny * (Nx - 1), x, y, Nx, y)
      end
    end
    if Ny > 1
      if y < Ny
        latt[b += 1] = LatticeBond(n, n + 1, x, y, x, y + 1)
      end
      if y == 1
        latt[b += 1] = LatticeBond(n, n + Ny - 1, x, y, x, Ny)
      end
    end
  end
  return latt
end

sqr_lat_pbc (generic function with 1 method)

In [89]:
Nx = 50
Ny = 50
J = 1
N = Nx*Ny
sites = siteinds("Qubit", N)
lat = sqr_lat_pbc(Nx, Ny)

H_os = OpSum()
for b in lat
    H_os += -J, "Z", b.s1, "Z", b.s2
end
H = MPO(H_os, sites);

state_Dict =  Dict(
    "All 0" => ["0" for i in 1:N],
    "Half 1" => [i <= round(Int, N/2) ? "0" : "1" for i in 1:N],
    "Alt 1/0" => [isodd(i) ? "1" : "0" for i in 1:N])
psi = MPS(sites, state_Dict["All 0"]);
expval = inner(psi', H, psi)

-5000.0